### Import

In [75]:
import os
import json
import requests
from bs4 import BeautifulSoup

### Extraction of opinion

In [76]:
with open('./cookie.json','r', encoding="UTF-8") as jf:
    headers = json.load(jf)

In [78]:
product_id = "17230358"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinions = []
while url:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, 'html.parser')
        opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        
        for opinion in opinions:
            single_opinion = {}
            single_opinion['opinion_id'] = opinion["data-entry-id"]
            single_opinion['author'] = opinion.select_one("span.user-post__author-name").text.strip()
            try:
                single_opinion['recommend'] = opinion.select_one("span.user-post__author-recomendation > em").text.strip()
            except AttributeError:
                single_opinion['recommend'] = None
            single_opinion['stars'] = opinion.select_one("span.user-post__score-count").text.strip()
            single_opinion['content'] = opinion.select_one("div.user-post__text").text.strip()
            single_opinion['pros'] = [tag.text.strip() for tag in opinion.select("div.review-feature__item--positive")]
            single_opinion['cons'] = [tag.text.strip() for tag in opinion.select("div.review-feature__item--negative")]
            single_opinion['up_votes'] = opinion.select_one("button.vote-yes")["data-total-vote"]
            single_opinion['down_votes'] = opinion.select_one("button.vote-no")["data-total-vote"]
            single_opinion['published'] = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
            try:
                single_opinion['purchased'] = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
            except TypeError:
                single_opinion['purchased'] = None
            all_opinions.append(single_opinion)
        try:
            url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            url = None
    print(all_opinions)

[{'opinion_id': '15931128', 'author': 'm...a', 'recommend': 'Polecam', 'stars': '5/5', 'content': 'Za te pieniądze jedna z lepszych na rynku. Co prawda poziom wykonania to nie jest Martin, ale jakość tej gitary kryje się w jej brzmieniu i wygodzie gry. To ten typ gitary, której nie chce się wypuszczać z rąk, i gra się dalej, aż do zmęczenia, które dzięki świetnej optymalizacji gryfu, akcji strun i pudła rezonansowego wcale tak szybko nie przyjdzie. Dla osób początkujących gitara na medal. Bardzo żywa, wyraźna dynamika. Dźwięk jest bardzo czytelny, pełny, śpiewny. Nie ma w niej tego "pudełkowatego" i płaskiego dołu i środka znanego z badziewnych tanich gitar. Poprawnie oddaje wszelkie zabiegi artykulacyjne, także czy to gra akordowa kostką czy fingerstyle, to i tak gra sprawia wiele radości. No to jest Yamaha, a w kwestii gitar ta firma trzyma poziom. Polecam i wracam do grania:)', 'pros': ['brzmienie', 'cena', 'jak dla mnie mało wygodna', 'jakość', 'jakość wykonania', 'ładnie pachnie',

In [79]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [80]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)